## **Extracting & Visualizing Stock Data - Tesla (TSLA) and GameStop (GME)**

<br>

### **Description**

<br>

In ths notebook, I will take historical stock information of Tesla and GameStop and plot it on a graph.

#### Setting up my environment
_Note: setting up my envirnoment by installing packages and loading libraries._

In [1]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Here, I'll define the function make_graph using a dataframe containing information about the stock, a dataframe containing information about the revenue, and the stock's name.

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

#### **Using yfinance to Extract Tesla Historical Stock Data**

Let's use the Ticker function to build a ticker object by entering the ticker symbol of the stock from which we wish to collect data. Tesla is the company's stock, and TSLA is its ticker.

In [3]:
tesla = yf.Ticker("TSLA")

Stock data will be extracted and saved in a dataframe called tesla_data using the ticker object and the function history. To obtain data for the longest possible period of time, we shall set the period option to max.

In [4]:
tesla_data = tesla.history(period="max")

The first five rows of the tesla_data dataframe will be displayed after we reset the index on the dataframe.

In [5]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


#### **Using Webscraping to Extract Tesla Revenue Data**

Using the requests library to download the webpage https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue and save the text of the response as a variable named html_data.

In [6]:
url_tsla = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data  = requests.get(url_tsla).text

We will Parse the html data using beautiful soup

In [7]:
soup_tsla = BeautifulSoup(html_data,"html5lib")

Using BeautifulSoup function, we will extract the table with Tesla Quarterly Revenue and store it into a dataframe named tesla_revenue showing the first 5 rows and removing the comma and dollar sign from the Revenue column.

In [8]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup_tsla.find_all('table'):

    if ('Tesla Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

        for row in rows:
            col = row.find_all('td')

            if col != []:
                date = col[0].text
                revenue = col[1].text.replace("$", "").replace(',','').replace('\n','')

                tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)

Executing the following lines to remove null or empty strings in the Revenue column.

In [10]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Displaying the last 5 row of the tesla_revenue dataframe using the tail function

In [11]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


#### **Using yfinance to Extract GameStop Historical Stock Data**

Again, let's use the Ticker function to enter the ticker symbol of GameStop and its ticker symbol is GME.

In [12]:
gamestop=yf.Ticker("GME")

Let's extract and save the gamestop data into a dataframe called gme_data using the ticker object and function history.

In [13]:
gme_data=gamestop.history(period="max")

As before, we will reset the index on the gamestop dataframe and display the first five rows.

In [14]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


#### **Using Webscraping to Extract GameStop Revenue Data**

Using the requests library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html and save the text of the response as a variable named html_data.

In [15]:
gme_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

html_data  = requests.get(gme_url).text

Now we will Parse the html data using beautiful soup

In [16]:
soup_gme = BeautifulSoup(html_data,"html5lib")

Using BeautifulSoup function, we will extract the table with GameStop Quarterly Revenue and store it into a dataframe named gme_revenue while removing the comma and dollar sign from the Revenue column.

In [17]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup_gme.find_all('table'):

    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

        for row in rows:
            col = row.find_all('td')

            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace("$", "").replace('\n','')


                gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)

Again, Executing the following lines to remove null or empty strings in the Revenue column.

In [18]:
gme_revenue.dropna(inplace=True)
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


Here we will use the make_graph function to graph the Tesla Stock Data.

_Note the graph will only show data upto June 2021._

In [19]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

<ipython-input-2-1df015be5a96>:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

<ipython-input-2-1df015be5a96>:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Here too, we will use the make_graph function to graph the GameStop Stock Data.

_Note the graph will only show data upto June 2021._

In [20]:
make_graph(gme_data, gme_revenue, 'GameStop')

<ipython-input-2-1df015be5a96>:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

<ipython-input-2-1df015be5a96>:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

